In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib import rc
import matplotlib.animation as animation
import numpy as np
from IPython.display import HTML, display, Video
import matplotlib.patches as patches
from copy import copy
from matplotlib.gridspec import GridSpec
import cloud_func_lib as cfl

font = {'family':'sans-serif', 'size'   : 18}
rc('font', **font)

In [7]:
def plot_summary(ds, key):
    fig = plt.figure(figsize=(30,11))
    gs1 = GridSpec(2,4,left=0.05,bottom=0.525,top=0.93,right=0.9,hspace=0.2,wspace=0.2) 
    axes1 = {}
    axes1['wp'] = fig.add_subplot(gs1[0])
    axes1['wp'].xaxis.set_visible(False)
    axes1['sp'] = fig.add_subplot(gs1[1])
    axes1['sp'].xaxis.set_visible(False)
    axes1['pre'] = fig.add_subplot(gs1[2])
    axes1['pre'].xaxis.set_visible(False)
    axes1['rad'] = fig.add_subplot(gs1[3])
    axes1['rad'].xaxis.set_visible(False)
    axes1['hcf'] = fig.add_subplot(gs1[4])
    axes1['tke'] = fig.add_subplot(gs1[5])
    axes1['sf'] = fig.add_subplot(gs1[6])
    axes1['we'] = fig.add_subplot(gs1[7])

    gs2 = GridSpec(2, 4,left=0.05,bottom=0.05,top=0.45,right=0.95,hspace=0.2,wspace=0.1)
    axes2 = {}
    axes2['vcf'] = fig.add_subplot(gs2[0])
    axes2['vcf'].xaxis.set_visible(False)
    axes2['u'] = fig.add_subplot(gs2[1])
    axes2['u'].xaxis.set_visible(False)
    axes2['u'].yaxis.set_visible(False)
    axes2['v'] = fig.add_subplot(gs2[2])
    axes2['v'].xaxis.set_visible(False)
    axes2['v'].yaxis.set_visible(False)
    axes2['w'] = fig.add_subplot(gs2[3])
    axes2['w'].xaxis.set_visible(False)
    axes2['w'].yaxis.set_visible(False)
    axes2['th'] = fig.add_subplot(gs2[4])
    axes2['dqv'] = fig.add_subplot(gs2[5])
    axes2['dqv'].yaxis.set_visible(False)
    axes2['dqc'] = fig.add_subplot(gs2[6])
    axes2['dqc'].yaxis.set_visible(False)
    axes2['dqr'] = fig.add_subplot(gs2[7])
    axes2['dqr'].yaxis.set_visible(False)

    vwp = ds.VWP_mean
    lwp = ds.LWP_mean*1000  #  g kg-1
    rwp = ds.RWP_mean*1000  # 

    #print(vwp, lwp, rwp)
    vwp_line, = vwp.plot(ax=axes1['wp'], c='C0', label='VWP')
    lwp_line, = lwp.plot(ax=axes1['wp'], c='C1', label='LWP')
    rwp_line, = rwp.plot(ax=axes1['wp'], c='C2', label='RWP')
    #a1.set_yscale('log')
    axes1['wp'].set_ylabel('(g kg^-1)')
    axes1['wp'].legend([vwp_line, lwp_line, rwp_line],['VWP','LWP','RWP'],loc=1)
    axes1['wp'].set_ylim(-1,300)
    cfl.add_diurnal(ds, axes1['wp'], (-1,300))

    surface_precip=ds.surface_precip_mean*1000
    time_ints=[ds.time_fine[i]-ds.time_fine[i-1] for i in range(1,len(ds.time_fine))]
    time_ints.insert(0,ds.time_fine[0].values)
    sp=[surface_precip[i]/time_ints[i] for i in range(len(surface_precip))]
    axes1['sp'].plot(ds.time_fine, sp)
    axes1['sp'].set_ylabel("Surf. pre. (mm hr^-1)")
    axes1['sp'].set_ylim(-0.01,0.2)
    cfl.add_diurnal(ds, axes1['sp'], (-0.01,0.2))
    
    ds.praut_mean.mean(axis=1).plot(label="autoconv",ax=axes1['pre'])
    ds.pracw_mean.mean(axis=1).plot(label="accretion",ax=axes1['pre'])
    ds.prevp_mean.mean(axis=1).plot(label="rain evap",ax=axes1['pre'])
    axes1['pre'].legend()
    axes1['pre'].set_xlabel("")
    axes1['pre'].set_ylabel("(kg kg^-1 s^-1)")
    axes1['pre'].set_ylim(-1e-10,1.5e-8)
    cfl.add_diurnal(ds, axes1['pre'], (-1e-9, 2e-8))
    
    cfl.sw_rad(axes1['rad'], ds, False, xlabel=True, location=(1.05,0))
    axes1['rad'].set_xlabel("")
    axes1['rad'].set_ylabel("(W m^-2)")

    cloud_frac = np.loadtxt(f"/home/users/eers/sct/lwp_mask_csvs/sct_{key}_cloud_frac.csv", delimiter=',')
    times = np.loadtxt(f"/home/users/eers/sct/lwp_mask_csvs/sct_{key}_times.csv", delimiter=',')
    axes1['hcf'].plot(times, cloud_frac)
    axes1['hcf'].set_ylim(0,1.05)
    cf_max = np.argmax(cloud_frac)
    if cf_max!=len(cloud_frac)-1:
        axes1['hcf'].plot((ds.time_coarse[np.argmax(np.abs(np.gradient(ds.surface_down_LW_mean[cf_max:])))],
                           ds.time_coarse[np.argmax(np.abs(np.gradient(ds.surface_down_LW_mean[cf_max:])))]), (0,1), c='C1')
    cfl.add_diurnal(ds, axes1['hcf'], (-0.01,1.1))
    axes1['hcf'].set_ylabel("Cloud frac")
    
    tke = ds.reske_mean + ds.subke_mean
    tke.plot(ax=axes1['tke'])
    axes1['tke'].set_ylabel("TKE (m^2 s^-2)")
    axes1['tke'].set_ylim(-0.01,1)
    cfl.add_diurnal(ds, axes1['tke'], (-0.01,1))
    axes1['tke'].set_xlabel("")

    ds.senhf_mean.plot(label='Sensible',c='C0', ax=axes1['sf'])
    ds.lathf_mean.plot(label='Latent',c='C1', ax=axes1['sf'])
    axes1['sf'].set_ylabel("Surf. flx (W m^-2)")
    axes1['sf'].set_yscale('log')
    axes1['sf'].legend()
    axes1['sf'].set_ylim(1,200)
    cfl.add_diurnal(ds, axes1['sf'], (-0.01,200))
    axes1['sf'].set_xlabel("")

    ds.w_min.plot(ax=axes1['we'], label="w min")
    ds.w_max.plot(ax=axes1['we'], label="w max")
    axes1['we'].set_xlabel("")
    axes1['we'].set_ylabel("(m s^-1)")  
    axes1['we'].set_ylim(-6,6)
    axes1['we'].legend(loc=(1.05,0))
    cfl.add_diurnal(ds, axes1['we'], (-6,6))

    ### g2
    
    clbas_ave, cltop_ave = cfl.cloud_bounds_bas_top(ds)
    clbase_thresh_heights = cfl.subcloud_thresh_timeseries(ds)
    for ax_key,ax_val in axes2.items():
        if ax_key=='vcf':
            c='white'
            a=1
        else:
            c='black'
            a=0.5
        clbas_ave.plot(color=c, label="Mean cloud base", ax=ax_val, alpha=a)
        cltop_ave.plot(color=c, label="Mean cloud top", ax=ax_val, alpha=a)
        
        inversion_heights1 = np.loadtxt(f"/home/users/eers/sct/lwp_mask_csvs/sct_{key}_inv_height.csv", delimiter=',')
        inversion_heights2 = np.loadtxt(f"/home/users/eers/sct/lwp_mask_csvs/sct_{key}_inv_height_deltheta.csv", delimiter=',')
        inversion_times1 = np.loadtxt(f"/home/users/eers/sct/lwp_mask_csvs/sct_{key}_times.csv", delimiter=',')
        inversion_times2 = np.loadtxt(f"/home/users/eers/sct/lwp_mask_csvs/sct_{key}_times_mid.csv", delimiter=',')
        ax_val.plot(inversion_times1, inversion_heights1, color=c, alpha=a, linestyle='--')
        ax_val.plot(inversion_times2, inversion_heights2, color='cyan', alpha=a, linestyle='--')
        ax_val.plot(inversion_times2, clbase_thresh_heights, color='cyan', alpha=a, linestyle='--')

    cf_ver=ds.total_cloud_fraction.plot(ax=axes2['vcf'], y='zn')
    cf_ver.colorbar.set_label("cloud frac")

    axes2['vcf'].set_ylabel("")
    axes2['vcf'].set_xlabel("")
    #a2.set_ylabel("")

    u_wind=ds.u_wind_mean.plot(y='zn', ax=axes2['u'])
    u_wind.colorbar.set_label("u (m s^-2)", wrap=True)
    axes2['u'].set_xlabel("")
    v_wind=ds.v_wind_mean.plot(y='zn', ax=axes2['v'])
    v_wind.colorbar.set_label("v (m s^-2)")
    axes2['v'].set_xlabel("")
    w_wind=ds.w_wind_mean.plot(y='z', ax=axes2['w'])
    w_wind.colorbar.set_label("w (m s^-2)")
    axes2['w'].set_xlabel("")

    th_mean=ds.theta_mean.plot(y='zn', ax=axes2['th'])
    th_mean.colorbar.set_label("theta (K)")
    axes2['th'].set_xlabel("")
    axes2['th'].set_ylabel("")
    
    dqv=ds.dqv_mphys_mean.plot(y='zn',ax=axes2['dqv'])
    dqv.colorbar.set_label("dqv")
    axes2['dqv'].set_xlabel("")
    axes2['dqv'].set_ylabel("")
    dqc=ds.dqc_mphys_mean.plot(y='zn',ax=axes2['dqc'])
    dqc.colorbar.set_label("dqc")
    axes2['dqc'].set_xlabel("")
    dqr=ds.dqr_mphys_mean.plot(y='zn',ax=axes2['dqr'])
    dqr.colorbar.set_label("dqr")
    axes2['dqr'].set_xlabel("")

    fig.text(0.42, 0.001,"Time (hours from 8LT)", ha='center')
    fig.text(0.01, 0.25, "Height (m)",  ha='center',rotation=90)
    fig.suptitle(f"{key}", x=0.14)
    fig.savefig(f"summary_plots/{key}_summary.png")
    plt.close()
    
    return ds

In [8]:
list_range=range(0,61)
design="em"

for n in list_range:
    key = f'{design}{str(n)}'
    ds_load = xr.open_dataset(f'/gws/nopw/j04/carisma/eers/sct/{design}/{key}/sct_{key}_merged.nc')
    ds = cfl.ds_fix_dims(ds_load)
    ds = plot_summary(ds, key)
    ds.close()
    